# American Options

## Black-scholes

In [46]:
###Cox Ross Rubinstein  binomial tree   

In [47]:
import time
import pandas as pd
import numpy as np
def measure_time(func, *args, **kwargs):
    start_time = time.time()  
    result = func(*args, **kwargs) 
    end_time = time.time()  
    execution_time = end_time - start_time  # Calculate the time difference
    return result, execution_time
# Cox Ross Rubinstein  binomial tree   
def crr(S0, K, T, r, sigma, n):

    dt = T / n  
    u = np.exp(sigma * np.sqrt(dt))  
    d = np.exp(-sigma * np.sqrt(dt)) 
    p = (np.exp(r * dt) - d) / (u - d)  
    

    asset_prices = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        for i in range(j + 1):
            asset_prices[i, j] = S0 * (u ** (j - i)) * (d ** i)
    

    option_prices = np.zeros((n + 1, n + 1))
    

    for i in range(n + 1):
        option_prices[i, n] = max(0, K-asset_prices[i, n] )
    
  
    for j in range(n - 1, -1, -1):
        for i in range(j + 1):
            option_prices[i, j] = np.exp(-r * dt) * (p * option_prices[i, j + 1] + (1 - p) * option_prices[i + 1, j + 1])
            
            option_prices[i, j] = max(option_prices[i, j], K-asset_prices[i, j] )

    return option_prices[0, 0]

### Partial differential equation

In [49]:
import numpy as np
from scipy.linalg import lu
from scipy.interpolate import interp1d, interp2d
from scipy import sparse
from scipy.sparse.linalg import splu
from scipy.sparse.linalg import spsolve
#Partial differential equation solver to American
#theta=0:explicit, theta=1 implicit, t=0.5 Crank-Nicolson method
def pde(S0, K, r,q, Time, sigma, M, N,theta,Smax):

    matval = np.zeros((M + 1, N + 1))
    S, dS = np.linspace(0, Smax, M + 1, retstep=True)  # space discretization
    T, dt = np.linspace(0, Time, N + 1, retstep=True)  # time discretization
    dSS=dS*dS
    j=T/dt
    # set up boundary conditions
    U_0 = np.maximum( K-S, 0)
    matval[:, N] = U_0
    matval[0, :] = K * np.exp(-r * (Time-dt * (j)))
    matval[M, :] = 0
    a=np.zeros(M+1)
    b=np.zeros(M+1)
    c = np.zeros(M + 1)
    for i in range(M+1):
        a[i] = 0.5 * (sigma** 2) * (S[i] **2);
        b[i] = (r - q) * S[i];
        c[i] = -r;
    alpha = (dt / dSS) * a[1:M] - dt  / (2 * dS) * b[1:M];
    beta = -2 * (dt / dSS) * a[1:M] + dt  * c[1:M];
    gamma =  (dt / dSS) * a[1:M] + dt  / (2 * dS) * b[1:M];
    D = sparse.diags([alpha[1:], beta, gamma[:M-1]], [-1, 0, 1], shape=(M-1, M-1), format='csr')
    I=sparse.eye(M-1, format='csr')
    A=I-theta*D
    B=I+(1-theta)*D
    aux1 = np.zeros(M-1)
    aux2 = np.zeros(M - 1)
    Array=np.zeros(N+1)
    lamda=U_0[1:M,]*0
    for j in range(N - 1, -1, -1):
        aux1[M-2]=-theta*gamma[-1]* matval[M,j+1]
        aux1[0] = -theta*alpha[0] * matval[0, j+1]
        aux2[M - 2] = (1-theta) * gamma[-1] * matval[M, j ]
        aux2[0] = (1-theta) * alpha[0] * matval[0, j ]
        matval[1:M, j] = spsolve(A, B@matval[1:M, j + 1]-aux1+aux2+dt*lamda)
        lamda_=lamda
        lamda = np.maximum(0, lamda_ + (U_0[1:M,] - matval[1:M, j]) / dt)
        matval[1:M, j] = np.maximum( matval[1:M, j] - dt* lamda_, U_0[1:M,]) 
    f=interp1d(S, matval[:,0], kind='linear', fill_value='extrapolate')    
    return f(S0) 

### trinomial tree

In [52]:
class TrinomialAmericanVanillaOpt:

    def __init__(self, spot, strike, rateD, time, volatility, rateF, optionType,
                 simulation):
        self.spot = spot
        self.strike = strike
        self.rateD = rateD
        self.time = time
        self.volatility = volatility
        self.rateF = rateF
        self.optionType = optionType
        self.simulation = simulation

    def get_spot(self):
        return self.spot

    def set_spot(self, spot):
        self.spot = spot

    def get_strike(self):
        return self.strike

    def set_strike(self, strike):
        self.strike = strike

    def get_rate_d(self):
        return self.rateD

    def set_rate_d(self, rate_d):
        self.rateD = rate_d

    def get_time(self):
        return self.time

    def set_time(self, time):
        self.time = time

    def get_volatility(self):
        return self.volatility

    def set_volatility(self, volatility):
        self.volatility = volatility

    def get_rate_f(self):
        return self.rateF

    def set_rate_f(self, rate_f):
        self.rateF = rate_f

    def get_option_type(self):
        return self.optionType

    def set_option_type(self, option_type):
        self.optionType = option_type

    def get_simulation(self):
        return self.simulation

    def set_simulation(self, simulation):
        self.simulation = simulation

    def put_trinomial(self, N):
        b = self.rateD - self.rateF
        dt = self.time / N
        u = np.exp(self.volatility * np.sqrt(2.0 * dt))
        d = 1.0 / u
        pu = ((np.exp(0.5 * b * dt) - np.exp(-self.volatility * np.sqrt(0.5 * dt))) /
              (np.exp(self.volatility * np.sqrt(0.5 * dt)) - np.exp(-self.volatility * np.sqrt(0.5 * dt)))) ** 2
        pd = ((np.exp(self.volatility * np.sqrt(0.5 * dt)) - np.exp(0.5 * b * dt)) /
              (np.exp(self.volatility * np.sqrt(0.5 * dt)) - np.exp(-self.volatility * np.sqrt(0.5 * dt)))) ** 2
        pm = 1.0 - pu - pd


        S = np.zeros((2 * N + 1, N + 1))
        V = np.zeros((2 * N + 1, N + 1))
        for j in range(N + 1):
            for i in range(2 * j + 1):
                S[i, j] = self.spot * u ** (j - i)

        V[:, N] = np.maximum(self.strike - S[:, N], 0.0)

        for j in range(N - 1, -1, -1):
            for i in range(2 * j + 1):
                V[i, j] = np.maximum(self.strike - S[i, j],
                                     np.exp(-self.rateD * dt) * (
                                                 pu * V[i, j + 1] + pm * V[i + 1, j + 1] + pd * V[i + 2, j + 1]))

        return V[0, 0]

    def call_trinomial(self, N):
        b = self.rateD - self.rateF
        dt = self.time / N
        u = np.exp(self.volatility * np.sqrt(2.0 * dt))
        d = 1.0 / u
        pu = ((np.exp(0.5 * b * dt) - np.exp(-self.volatility * np.sqrt(0.5 * dt))) /
              (np.exp(self.volatility * np.sqrt(0.5 * dt)) - np.exp(-self.volatility * np.sqrt(0.5 * dt)))) ** 2
        pd = ((np.exp(self.volatility * np.sqrt(0.5 * dt)) - np.exp(0.5 * b * dt)) /
              (np.exp(self.volatility * np.sqrt(0.5 * dt)) - np.exp(-self.volatility * np.sqrt(0.5 * dt)))) ** 2
        pm = 1.0 - pu - pd
    

        S = np.zeros((2 * N + 1, N + 1))
        V = np.zeros((2 * N + 1, N + 1))
        for j in range(N + 1):
            for i in range(2 * j + 1):
                S[i, j] = self.spot * u ** (j - i)

        V[:, N] = np.maximum(S[:, N] - self.strike, 0.0)

        for j in range(N - 1, -1, -1):
            for i in range(2 * j + 1):
                V[i, j] = np.maximum(S[i, j] - self.strike,
                                     np.exp(-self.rateD * dt) * (
                                                 pu * V[i, j + 1] + pm * V[i + 1, j + 1] + pd * V[i + 2, j + 1]))
   
        return V[0, 0]

    def evaluate(self):
        if self.optionType == 1:
            return self.call_trinomial(self.simulation)
        else:
            return self.put_trinomial(self.simulation)


In [53]:
def triniomal_American( spot, strike, rateD, time, volatility, rateF, optionType,simulation):
    price=TrinomialAmericanVanillaOpt( spot, strike, rateD, time, volatility, rateF, optionType,simulation)
    return price.evaluate()
     

### Monte-Carlo

In [60]:
def longstaff(S0,K,T,r,sigma):
    N=300
    M=200000
    dt = T / (N - 1)  # time step
    df = np.exp(-r * dt)
    def GBM_Process(maturity, nSims, drift, spot_price, sigma, dt):
        spotsims = np.zeros((  maturity,nSims))
        spotsims[0, :] = spot_price
        for t in range(1, maturity):
            z = np.random.standard_normal(nSims)
            spotsims[t, :] = spotsims[t-1, :] * np.exp((drift - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
        return spotsims
    S=GBM_Process(N, M, r, S0, sigma, dt)
    def lsm(S,K,r,t):
        
        N=S.shape[0]
        dt = T / (N - 1)

        CF = np.zeros_like(S)
        CF[-1, :] = np.maximum(K - S[-1, :], 0)


        for ii in range(N - 2, 0, -1):
            idx = np.where(S[ii, :] < K)[0]
    
    
            X = S[ii, idx]
            NX=len(X)
            X1 = X / S0
            Y = CF[ii + 1, idx] * np.exp(-r * dt)
    
            # Linear regression step
            R = np.vstack([np.ones_like(X1), (1 - X1), 1/2 * (2 - 4 * X1 - X1**2)])
            a = np.linalg.lstsq(R.T, Y, rcond=None)[0]
    
            C = R.T @ a
    
            jdx = np.where(np.maximum(K - X, 0) > C)[0]
            nidx = np.setdiff1d(np.arange(M), idx[jdx])
    
            CF[ii, idx[jdx]] = np.maximum(K - X[jdx], 0)
            CF[ii, nidx] = np.exp(-r * dt) * CF[ii + 1, nidx]

        return np.mean(CF[1, :]) * np.exp(-r * dt)

    return lsm(S,K,r,T)

In [61]:
S0=100
K=100
T=0.5
r=0.02
sigma=0.15
n=1000

args_f = (S0, K, T, r, sigma, n)  
args_g = (S0, K, r,0, T, sigma, 1000, 300,0.5,1000)  
args_t = ( S0, K, r, T, sigma, 0, -1 ,200)
args_h = (S0,K,T,r,sigma) 


results = {
    'Function': ['crr', 'pde', 'trinomial', 'longstaff'],
    'Value': [],
    'Execution Time (s)': []
}


value_f, exec_time_f = measure_time(crr, *args_f)
value_g, exec_time_g = measure_time(pde, *args_g)
value_t, exec_time_t = measure_time(triniomal_American, *args_t)
value_h, exec_time_h = measure_time(longstaff, *args_h)


results['Value'].append(value_f)
results['Execution Time (s)'].append(exec_time_f)

results['Value'].append(value_g)
results['Execution Time (s)'].append(exec_time_g)
results['Value'].append(value_t)
results['Execution Time (s)'].append(exec_time_t)
results['Value'].append(value_h)
results['Execution Time (s)'].append(exec_time_h)


df = pd.DataFrame(results)


print(df)

    Function               Value  Execution Time (s)
0        crr           10.570537            2.787762
1        pde  10.567255142897979            0.221693
2  trinomial           10.568659            0.288703
3  longstaff           10.551552           16.830700


In [59]:
S0=100
K=110
T=0.5
r=0.02
sigma=0.15
n=1000
args_f = (S0, K, T, r, sigma, n)  
args_g = (S0, K, r,0, T, sigma, 1000, 300,0.5,200)  
args_t = ( S0, K, r, T, sigma, 0, -1 ,200)
args_h = (S0,K,T,r,sigma) 
results = {
    'Function': ['crr', 'pde', 'trinomial', 'longstaff'],
    'Value': [],
    'Execution Time (s)': []
}


value_f, exec_time_f = measure_time(crr, *args_f)
value_g, exec_time_g = measure_time(pde, *args_g)
value_t, exec_time_t = measure_time(triniomal_American, *args_t)
value_h, exec_time_h = measure_time(longstaff, *args_h)


results['Value'].append(value_f)
results['Execution Time (s)'].append(exec_time_f)

results['Value'].append(value_g)
results['Execution Time (s)'].append(exec_time_g)
results['Value'].append(value_t)
results['Execution Time (s)'].append(exec_time_t)
results['Value'].append(value_h)
results['Execution Time (s)'].append(exec_time_h)


df = pd.DataFrame(results)


print(df)

    Function              Value  Execution Time (s)
0        crr          10.570537            2.471643
1        pde  10.57030434175276            0.220584
2  trinomial          10.568659            0.251621
3  longstaff          10.536285           17.351962
